# Photo Classifier: Model evaluation

## Imports

In [1]:
import random
import numpy as np
import pandas as pd
from collections import Counter
from ipywidgets import interact
import ipywidgets as widgets

In [2]:
from file_io import load_pickle_file
from constants import MODEL_FILE_PATH
from image_transforms import IMAGE_TRANSFORMS
from prediction import predict
from interpreter import Interpreter

In [3]:
import random
random.seed(42)

In [4]:
DATA_SET = 'test'

## Load model and data

In [5]:
model_data = load_pickle_file(MODEL_FILE_PATH)
model = model_data['model']
image_data = model_data['data']

## Prediction

In [6]:
x = image_data.get_images(DATA_SET)
y_true = image_data.get_classes(DATA_SET)

In [7]:
y_pred, probabilities = predict(x, model, IMAGE_TRANSFORMS[DATA_SET])

## Results interpretation

In [8]:
interpreter = Interpreter(x, y_pred, y_true, probabilities, image_data.class_to_label_mapping)

In [9]:
accuracy = interpreter.calculate_accuracy()
accuracy

0.9

In [10]:
confusion_matrix = interpreter.calculate_confusion_matrix()
confusion_matrix

Prediction,me,not_people,other_people
True,,,
me,9,1,0
not_people,0,10,0
other_people,1,1,8


In [11]:
accuracy_by_label = interpreter.calculate_accuracy_by_label()
accuracy_by_label

,is_correct
label,
me,0.9
not_people,1.0
other_people,0.8


In [12]:
misclassified_samples = interpreter.get_misclassified_samples()
misclassified_samples

,true_label,predicted_label,propability_true,propability_predicted,is_correct
0,other_people,me,0.182219,0.642801,False
19,other_people,not_people,0.190245,0.752650,False
28,me,not_people,0.309437,0.647361,False


In [13]:
most_uncertain_samples = interpreter.get_most_uncertain_samples(5)
most_uncertain_samples

,probability
sample,
0,0.642801
28,0.647361
17,0.688382
24,0.714171
3,0.715679


In [14]:
most_incorrect_samples = interpreter.get_most_incorrect_samples(5)
most_incorrect_samples

,probability
sample,
0,0.182219
19,0.190245
28,0.309437
17,0.688382
24,0.714171


In [15]:
@interact(index=widgets.IntSlider(min=0, max=len(y_true)-1, step=1, value=0, continuous_update=False))
def show_prediction(index=0):
    interpreter.plot_prediction(index)

interactive(children=(IntSlider(value=0, continuous_update=False, description='index', max=29), Output()), _do…